## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [2]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [3]:
reference_speaker = './resources/c-div-seagull.wav' # This is the voice you want to clone
# reference_speaker = './resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v2


c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\torch\functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\SpectralOps.cpp:878.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

In [4]:
from melo.api import TTS

texts = {
    'EN_NEWEST': "Hello!. Welcome to Nottingham Trent University’s Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham, Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.",  # The newest English base speaker model
    'EN': "Hello!. Welcome to Nottingham Trent University’s Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham, Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.",  # The newest English base speaker model
    # 'EN': "Did you ever hear a folk tale about a giant turtle?",
    # 'EN_NEWEST': "Did you ever hear a folk tale about a giant turtle?",
    # 'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    # 'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    # 'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    # 'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    # 'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}


src_path = f'{output_dir}/clean-seagull-temp.wav'

# Speed is adjustable
speed = 1

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    print(speaker_ids)
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_clean_seagull_fast_{language}_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)

c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


{'EN-Newest': 0}
 > Text split to sentences.
Hello! . Welcome to Nottingham Trent University's Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham,
Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.
 > ===========================


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\ddcdi\Docume

{'EN-US': 0, 'EN-BR': 1, 'EN_INDIA': 2, 'EN-AU': 3, 'EN-Default': 4}
 > Text split to sentences.
Hello! . Welcome to Nottingham Trent University's Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham,
Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.
 > ===========================


100%|██████████| 2/2 [00:00<00:00,  4.69it/s]


...

100%|██████████| 2/2 [00:00<00:00,  4.75it/s]


 > Text split to sentences.
Hello! . Welcome to Nottingham Trent University's Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham,
Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.
 > ===========================


100%|██████████| 2/2 [00:00<00:00,  5.04it/s]


 > Text split to sentences.
Hello! . Welcome to Nottingham Trent University's Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham,
Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.
 > ===========================


100%|██████████| 2/2 [00:00<00:00,  4.70it/s]
